## Chroma demo操作

官方範例參考: https://docs.trychroma.com/getting-started

1. Install chroma
2. Create the client
3. Create the collection
4. Load data
5. Query data by nearest embedding

<img src="https://www.trychroma.com/hrm4.svg"  width="640" height="320">

In [1]:
!nvidia-smi

Fri Dec 22 12:47:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝 chroma

In [2]:
!pip install chromadb==0.4.20 datasets==2.15.0 numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 k

In [3]:
#在2023/12/18測試時發現chromadb有一些相容性問題,需手動安裝降版的相關套件. *之後官方應該會解決。

!pip install kaleido python-multipart tensorflow-probability==0.20.0 typing-extensions==4.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.9 MB/s eta 0:00:00
Reason for being yanked: TFP 0.20 should require Python 3.8+
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.22.0
    Uninstalling tensorflow-probability-0.22.0:
      Successfully uninstalled tensorflow-probability-0.22.0


### 建立 client & collection

- 建立local檔案資料庫(sqlite3): chromadb.PersistentClient(path="存檔路徑")
- 不建立檔案，只在記憶體操作: chromadb.Client()

In [4]:
import chromadb

#In-memory chroma with saving/loading to disk
client = chromadb.PersistentClient(path="./chromadb/")

#client = chromadb.Client() #這個是不建立local檔案(sqlite3)的操作模式

collection = client.get_or_create_collection("test") #這個呼叫方法不會重置己有的db

In [5]:
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

### 載入資料集 (使用load_dataset方法)

我們使用[SciQ資料集](https://arxiv.org/abs/1707.06209)，它可以在[HuggingFace](https://huggingface.co/datasets/sciq)上找到。

以下是來自HuggingFace的資料集描述：

> SciQ數據集包含13,679個由眾包方式收集的科學考試問題，涵蓋物理、化學、生物學等多個科目。這些問題採用多選擇格式，每個問題有4個選擇項。對於大多數問題，還提供了一段額外的支持正確答案的證據段落。


In [6]:
# Get the SciQ dataset from HuggingFace
from datasets import load_dataset

dataset = load_dataset("sciq", split="train")

# Filter the dataset to only include questions with a support
dataset = dataset.filter(lambda x: x["support"] != "")

print("Number of questions with support: ", len(dataset))

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

Number of questions with support:  10481


Chroma comes with a built-in embedding model, which makes it simple to load text.
We can load the SciQ dataset into Chroma with just a few lines of code.

In [7]:
# Create a new Chroma collection to store the supporting evidence. We don't need to specify an embedding fuction, and the default will be used.
collection = client.create_collection("sciq_supports")


In [8]:
# Embed and store the first 100 supports for this demo
collection.add(
    ids=[str(i) for i in range(0, 100)],  # IDs are just strings
    documents=dataset["support"][:100],
    metadatas=[{"type": "support"} for _ in range(0, 100)
    ],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 24.7MiB/s]


### 資料查詢(Query data)
Chroma支援多種查詢方式，透過embedding similarity score來進行資料查詢的工作。

In [9]:
results = collection.query(
    query_texts=dataset["question"][:10],
    n_results=1)

we display the query questions along with their retrieved supports

In [12]:
# Print the question and the corresponding support
for i, q in enumerate(dataset['question'][:10]):
    print(f"Question: {q}")
    print(f"Retrieved support: {results['documents'][i][0]}")
    print()

Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Retrieved support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Retrieved support: Without Coriolis Effect the global winds would blow north to south



---



## 簡易向量資料庫操作

查詢目前己有的DB(Collection)

In [13]:
client.list_collections(), client.get_version()

([Collection(name=sciq_supports), Collection(name=test)], '0.4.20')

In [14]:
import chromadb
client = chromadb.Client()

collection = client.get_or_create_collection("my_stock_list") #這個呼叫方法不會重置己有的db


# prompt: 如何查詢指定的collection是否存在?

collection = client.get_collection("my_stock_list")

print("collection_name: ",collection.name)
print("collection_tenant: ",collection.tenant)

assert collection.id == collection.id
assert collection.tenant == collection.tenant
assert collection.name == collection.name
assert collection.database == collection.database

print("collection_database: ",collection.database)

collection_name:  my_stock_list
collection_tenant:  default_tenant
collection_database:  default_database


### 初始化DB

In [15]:
### 以下是判斷"my_stock_list"是否己存在,若己存在就先刪掉.

try:
    collection = client.get_collection("my_stock_list")

    if not collection is None:
      client.delete_collection(name="my_stock_list")
except:
    print("指定的collection('my_stock_list')不存在,需重新建立!")


In [16]:
import chromadb
chroma_client = chromadb.Client()

# 起一個新集合名字为"my_stock_list"
collection = chroma_client.create_collection(name="my_stock_list")

In [17]:
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

### 新增 (使用add方法)

In [18]:
# 新增資料庫資料記錄，利用embeddings為資料記錄指定文本(記錄)的代表向量

collection.add(
    embeddings=[
        [0.17652, 1.0, 1.0],
        [0.585, 1.0, 2.0],
        [0.1730, 1.0, 2.0],
        [0.1725, 1.0, 2.0],
        [0.23341, 2.0, 1.0],
        [0.626, 2.0, 2.0],
        [0.2465, 2.0, 2.0],
        [0.1205, 2.0, 2.5],
        [0.3777, 1.0, 3.0],
    ],
    metadatas=[
        {"name": "加權指數", "type": "上市"},
        {"name": "台積電", "type": "上市"},
        {"name": "創意", "style": "上市"},
        {"name": "中砂", "style": "上市"},
        {"name": "櫃買指數", "style": "上櫃"},
        {"name": "環球晶", "style": "上櫃"},
        {"name": "力旺", "style": "上櫃"},
        {"name": "譜瑞-KY", "style": "上櫃"},
        {"name": "國泰費城半導體", "style": "上市"},
    ],
    documents=["doc1-上市", "doc2-上市", "doc3-上市", "doc4-上市", "doc5-上櫃", "doc6-上櫃", "doc7-上櫃", "doc8-上櫃", "doc9-上市"],
    ids=["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8", "id9"],
)

### 查詢

In [19]:
#查詢全部
collection.get(include=['embeddings', 'documents', 'metadatas'])

#查詢時如果沒有指明顯示項目,有些項目不會顯示,例如:embeddings

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9'],
 'embeddings': [[0.17652000486850739, 1.0, 1.0],
  [0.5849999785423279, 1.0, 2.0],
  [0.17299999296665192, 1.0, 2.0],
  [0.17249999940395355, 1.0, 2.0],
  [0.23341000080108643, 2.0, 1.0],
  [0.6259999871253967, 2.0, 2.0],
  [0.24650000035762787, 2.0, 2.0],
  [0.12049999833106995, 2.0, 2.5],
  [0.37770000100135803, 1.0, 3.0]],
 'metadatas': [{'name': '加權指數', 'type': '上市'},
  {'name': '台積電', 'type': '上市'},
  {'name': '創意', 'style': '上市'},
  {'name': '中砂', 'style': '上市'},
  {'name': '櫃買指數', 'style': '上櫃'},
  {'name': '環球晶', 'style': '上櫃'},
  {'name': '力旺', 'style': '上櫃'},
  {'name': '譜瑞-KY', 'style': '上櫃'},
  {'name': '國泰費城半導體', 'style': '上市'}],
 'documents': ['doc1-上市',
  'doc2-上市',
  'doc3-上市',
  'doc4-上市',
  'doc5-上櫃',
  'doc6-上櫃',
  'doc7-上櫃',
  'doc8-上櫃',
  'doc9-上市'],
 'uris': None,
 'data': None}

Query data by nearest embedding

In [20]:
#條件式查詢
collection.query(
    query_embeddings=[[0.5, 1.0, 1.0], [0.17, 2.1, 1.7]], #給兩組向量進行查詢
    n_results=2,                        #每一組查詢向量回傳2個最接近的結果
    #where={"metadata_field": "my_source"},
    where_document={"$contains":"doc"}
)

{'ids': [['id1', 'id2'], ['id7', 'id6']],
 'distances': [[0.10463931411504745, 1.0072250366210938],
  [0.10585220158100128, 0.3079359233379364]],
 'metadatas': [[{'name': '加權指數', 'type': '上市'}, {'name': '台積電', 'type': '上市'}],
  [{'name': '力旺', 'style': '上櫃'}, {'name': '環球晶', 'style': '上櫃'}]],
 'embeddings': None,
 'documents': [['doc1-上市', 'doc2-上市'], ['doc7-上櫃', 'doc6-上櫃']],
 'uris': None,
 'data': None}

In [21]:
#依id做查詢
collection.get(ids=["id3","id4"])

#查詢時如果沒有指明顯示項目,有些項目不會顯示,例如:embeddings

{'ids': ['id3', 'id4'],
 'embeddings': None,
 'metadatas': [{'name': '創意', 'style': '上市'}, {'name': '中砂', 'style': '上市'}],
 'documents': ['doc3-上市', 'doc4-上市'],
 'uris': None,
 'data': None}

### 修改

In [22]:
#依id更新資料
collection.update(
    ids=["id3"], #依ids做修改
    embeddings=[[0.2100, 1.0, 2.0]],
    documents=["這是修改後的id3"],
)

In [23]:
collection.get("id3")

{'ids': ['id3'],
 'embeddings': None,
 'metadatas': [{'name': '創意', 'style': '上市'}],
 'documents': ['這是修改後的id3'],
 'uris': None,
 'data': None}

In [24]:
# 新增及修改 (upsert items. new items will be added, existing items will be updated.)
collection.upsert(
    ids=["id3"],
    embeddings=[[0.1730, 1.0, 2.0]],
    metadatas=[{'name': '創意', 'style': '上市'}],
    documents=['這是執行update&insert後的id3'],
)

In [25]:
collection.get("id3")

{'ids': ['id3'],
 'embeddings': None,
 'metadatas': [{'name': '創意', 'style': '上市'}],
 'documents': ['這是執行update&insert後的id3'],
 'uris': None,
 'data': None}

### 刪除

In [26]:
collection.delete(
    ids=["id1"],
    where={"name": "加權指數"} #where加上刪除的條件
)

In [27]:
collection.get() #確認一下是否有確實刪除

{'ids': ['id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9'],
 'embeddings': None,
 'metadatas': [{'name': '台積電', 'type': '上市'},
  {'name': '創意', 'style': '上市'},
  {'name': '中砂', 'style': '上市'},
  {'name': '櫃買指數', 'style': '上櫃'},
  {'name': '環球晶', 'style': '上櫃'},
  {'name': '力旺', 'style': '上櫃'},
  {'name': '譜瑞-KY', 'style': '上櫃'},
  {'name': '國泰費城半導體', 'style': '上市'}],
 'documents': ['doc2-上市',
  '這是執行update&insert後的id3',
  'doc4-上市',
  'doc5-上櫃',
  'doc6-上櫃',
  'doc7-上櫃',
  'doc8-上櫃',
  'doc9-上市'],
 'uris': None,
 'data': None}

### 其它操作

In [28]:
#chromadb環境設定
chromadb.config.Settings()

Settings(environment='', chroma_db_impl=None, chroma_api_impl='chromadb.api.segment.SegmentAPI', chroma_product_telemetry_impl='chromadb.telemetry.product.posthog.Posthog', chroma_telemetry_impl='chromadb.telemetry.product.posthog.Posthog', chroma_sysdb_impl='chromadb.db.impl.sqlite.SqliteDB', chroma_producer_impl='chromadb.db.impl.sqlite.SqliteDB', chroma_consumer_impl='chromadb.db.impl.sqlite.SqliteDB', chroma_segment_manager_impl='chromadb.segment.impl.manager.local.LocalSegmentManager', chroma_segment_directory_impl='chromadb.segment.impl.distributed.segment_directory.RendezvousHashSegmentDirectory', chroma_memberlist_provider_impl='chromadb.segment.impl.distributed.segment_directory.CustomResourceMemberlistProvider', chroma_collection_assignment_policy_impl='chromadb.ingest.impl.simple_policy.SimpleAssignmentPolicy', worker_memberlist_name='worker-memberlist', tenant_id='default', topic_namespace='default', is_persistent=False, persist_directory='./chroma', chroma_server_host=None

In [29]:
#列出前10個項目
collection.peek() # returns a list of the first 10 items in the collection

{'ids': ['id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9'],
 'embeddings': [[0.5849999785423279, 1.0, 2.0],
  [0.17299999296665192, 1.0, 2.0],
  [0.17249999940395355, 1.0, 2.0],
  [0.23341000080108643, 2.0, 1.0],
  [0.6259999871253967, 2.0, 2.0],
  [0.24650000035762787, 2.0, 2.0],
  [0.12049999833106995, 2.0, 2.5],
  [0.37770000100135803, 1.0, 3.0]],
 'metadatas': [{'name': '台積電', 'type': '上市'},
  {'name': '創意', 'style': '上市'},
  {'name': '中砂', 'style': '上市'},
  {'name': '櫃買指數', 'style': '上櫃'},
  {'name': '環球晶', 'style': '上櫃'},
  {'name': '力旺', 'style': '上櫃'},
  {'name': '譜瑞-KY', 'style': '上櫃'},
  {'name': '國泰費城半導體', 'style': '上市'}],
 'documents': ['doc2-上市',
  '這是執行update&insert後的id3',
  'doc4-上市',
  'doc5-上櫃',
  'doc6-上櫃',
  'doc7-上櫃',
  'doc8-上櫃',
  'doc9-上市'],
 'uris': None,
 'data': None}

In [30]:
#查詢目前資料的總筆數
collection.count() # returns the number of items in the collection

8

In [33]:
#查詢目前DB有那些collections
client.list_collections()

[Collection(name=test_1218)]

In [32]:
#更改collections的名稱 (名稱不能跟現有的collection重覆!)
collection.modify(name="test_1218") # Rename the collection

In [34]:
#查詢目前timestamp
client.heartbeat()

1703249899904544852

### 環境操作(command)
**這些操作在colab沒辨法執行**，相關DB server操作可以參考官網說明:
- https://docs.trychroma.com/deployment
- https://docs.trychroma.com/migration

In [ ]:
!chromadb --help

/bin/bash: line 1: chromadb: command not found


In [ ]:
#把目前DB的資料儲存到本地路徑
!chromadb save --path /path/to/save/collections

/bin/bash: line 1: chromadb: command not found




---



## 進階向量資料庫操作

In [35]:
!pip -q install chromadb==0.4.20 cohere tiktoken langchain openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00


In [36]:
#在2023/12/18測試時發現chromadb有一些相容性問題,需手動安裝降版的相關套件. *之後官方應該會解決。

!pip install kaleido python-multipart tensorflow-probability==0.20.0 typing-extensions==4.9.0

### 其它各種embedding的做法

Chroma - Embeddings說明: https://docs.trychroma.com/embeddings


#### 使用OpenAI Embeddings

##### 取得OpenAI API-KEY

In [ ]:
import os
import openai

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    # read from file
    with open('OPENAI_API_KEY.txt', 'r') as f:
        OPENAI_API_KEY = f.read().strip()

openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.Client()
# Using OpenAI Embeddings. This assumes you have the openai package installed
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"], # Replace with your own OpenAI API key
    model_name="text-embedding-ada-002"
)


In [ ]:
# Create a new chroma collection
openai_collection = client.get_or_create_collection(name="openai_embeddings", embedding_function=openai_ef)

In [ ]:
openai_collection.add(
    documents=["假設這是一篇公司的SOP文件說明", "假設這是一篇公司的公告", "假設這是一篇公司晨會的會議記錄"],
    metadatas=[{"source": "SOP-DB"}, {"source": "COM-DB"}, {"source": "COM-DB"}],
    ids=["id1", "id2", "id3"]
)

In [ ]:
results = openai_collection.query(
    query_texts=["查詢會議記錄"],
    n_results=2
)
results

{'ids': [['id3', 'id1']],
 'distances': [[0.2406662404537201, 0.5413336157798767]],
 'metadatas': [[{'source': 'COM-DB'}, {'source': 'my_source'}]],
 'embeddings': None,
 'documents': [['假設這是一篇公司晨會的會議記錄', 'This is a document']],
 'uris': None,
 'data': None}

#### 使用HuggingFace Embeddings

使用**all-MiniLM-L6-v2**

它是一個sentence-transformers模型: 會mapping sentences 及 paragraphs 到一個**384維度的向量空間**，可以用來做為分群或是計算向量相似度之應用。

**huggingface model name: `sentence-transformers/all-MiniLM-L6-v2`**

##### 登入Huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
HUGGINGFACE_API_KEY = "hf_XXXXXXXXXXXXXXXXXXXXXXXX" #有讀取權限的即可

In [ ]:
# Using HuggingFace models. The embedding function a huggingface api_key
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=HUGGINGFACE_API_KEY, # Replace with your own HuggingFace API key
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
### 以下是判斷"huggingface_embeddings"是否己存在,若己存在就先刪掉.

try:
    collection = client.get_collection("huggingface_embeddings")

    if not collection is None:
      client.delete_collection(name="huggingface_embeddings")
except:
    print("指定的collection('huggingface_embeddings')不存在,需重新建立!")


In [ ]:
# Create a new HuggingFace collection
huggingface_collection = client.create_collection(name="huggingface_embeddings", embedding_function=huggingface_ef)

In [ ]:
huggingface_collection.add(
    documents=["假設這是一篇公司的SOP文件說明", "假設這是一篇公司的公告", "假設這是一篇公司晨會的會議記錄"],
    metadatas=[{"source": "SOP-DB"}, {"source": "COM-DB"}, {"source": "COM-DB"}],
    ids=["id1", "id2", "id3"]
)

In [ ]:
results = huggingface_collection.query(
    query_texts=["查詢SOP文件"],
    n_results=2,
    #include=['embeddings'], #可以查看embeddings的值.(384維,很長)
)
results

{'ids': [['id1', 'id3']],
 'distances': [[0.7498685121536255, 1.1396143436431885]],
 'metadatas': [[{'source': 'SOP-DB'}, {'source': 'COM-DB'}]],
 'embeddings': None,
 'documents': [['假設這是一篇公司的SOP文件說明', '假設這是一篇公司晨會的會議記錄']],
 'uris': None,
 'data': None}



---



## 課堂練習


**請練習把huggingface的資料集，轉成chromadb的collection**

我們使用的資料集: `yentinglin/TaiwanChat`

Taiwan LLM: Bridging the Linguistic Divide with a Culturally Aligned Language Model

2023/10/20 (更新) 一些HF平台上的繁體中文資料集:
- Taiwan-LLaMa模型訓練資料集:(https://huggingface.co/datasets/yentinglin/traditional_mandarin_instructions)
- Erhwen,Kuo個人整理,一些知名的語料集之繁體中文化:(https://huggingface.co/erhwenkuo)
- 網頁新聞,由簡體轉繁體(文化對齊問題請自行評估):(https://huggingface.co/datasets/jed351/Traditional-Chinese-Common-Crawl-Filtered)
- TAIDE計畫的測試資料集:(https://huggingface.co/datasets/taide/TAIDE-14-tasks)

(參考)簡體中文資料集:
- 北京智源人工智能研究院-BAAI DataHub（data.baai.ac.cn）:(https://huggingface.co/datasets/BAAI/COIG-PC)

(參考) github平台上的資源:
- 聯發創新基地（MediaTek Research)(https://github.com/mtkresearch/MR-Models/tree/main)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yentinglin/TaiwanChat", split="train")

In [ ]:
#因教學示範使用: 先縮減資料集為100筆資料
dataset = dataset.select(range(100))

In [ ]:
def process_func(example):

    # 依来源资料集的格式，去凑出 instruction 及 response
    instruction = "Human: " + example['conversations'][0]['value'] + "\n\nAssistant: "

    if len(example['conversations']) == 1:  # 如果对话没有一问一答时候(len=2)
        response = "</s>"  # 给终止符号
    else:
        response = example['conversations'][1]['value'] + "</s>"

    return {
        "documents": instruction+response
    }

In [ ]:
source_ds = dataset.map(process_func, remove_columns=dataset.column_names)
source_ds

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['documents'],
    num_rows: 100
})

In [ ]:
import chromadb
client = chromadb.Client()
#client.delete_collection(name="TaiwanChat")

# Create a new Chroma collection
collection = client.create_collection("TaiwanChat")

In [ ]:
source_ds['documents'][:5]

['Human: 給我三個保持健康的小撇步。\n\nAssistant: 1.飲食要均衡，多攝取水果和蔬菜。\n2.定期運動，保持身體活躍和強壯。\n3.保持充足的睡眠，並保持一個固定的睡眠時間表。</s>',
 'Human: 三原色是什麼？\n\nAssistant: 三原色是紅色、藍色和黃色。</s>',
 'Human: 描述一個原子的結構。\n\nAssistant: 一個原子由核心組成，核心包含著負負電荷的質子和中性的中子，而圍繞在核心周圍的是帶有負電荷的電子。質子和中子具有正電荷，而電子具有負電荷，因此整個原子是中性的。每種粒子的數量決定了原子的原子序數和種類。</s>',
 'Human: 我們該如何減少空氣污染？\n\nAssistant: 有許多方法可以減少空氣污染，例如轉向使用可再生能源、鼓勵使用大眾運輸工具、禁止燃燒化石燃料、實施減少工業排放的政策，以及實施車輛排放標準。此外，個人也可以盡自己的一份力量減少空氣污染，例如減少開車、避免燃燒木材等物質，以及改用節能電器。</s>',
 'Human: 假設你是一家建築公司的專案經理，描述一個你必須做出困難決策的時刻。\n\nAssistant: 當我在一家建築公司擔任專案經理時，我曾經面臨過一個困難的決策。我負責的專案需要在特定日期前完成，以滿足客戶的期望。然而，由於意外的延誤，我們無法在截止日期前完成，因此我必須做出困難的決定。我決定延長截止日期，但這樣一來我必須進一步動用團隊的資源並增加預算。儘管這是一個冒險的決定，但我最終決定繼續進行，以確保專案能按時完成並滿足客戶的期望。最終，專案成功完成，這被視為我領導能力和決策能力的證明。</s>']

In [ ]:
# Embed and store the first 100 records for this demo
# 預設會用all-MiniLM-L6-v2來做embeding

collection.add(
    ids=[str(i) for i in range(0, 100)],  # IDs are just strings
    documents=source_ds['documents'][:100],
    metadatas=[{"type": "chat"} for _ in range(0, 100)
    ],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 11.3MiB/s]


In [ ]:
#列出前3個項目
collection.get(["0","1","2"])

{'ids': ['0', '1', '2'],
 'embeddings': None,
 'metadatas': [{'type': 'chat'}, {'type': 'chat'}, {'type': 'chat'}],
 'documents': ['Human: 給我三個保持健康的小撇步。\n\nAssistant: 1.飲食要均衡，多攝取水果和蔬菜。\n2.定期運動，保持身體活躍和強壯。\n3.保持充足的睡眠，並保持一個固定的睡眠時間表。</s>',
  'Human: 三原色是什麼？\n\nAssistant: 三原色是紅色、藍色和黃色。</s>',
  'Human: 描述一個原子的結構。\n\nAssistant: 一個原子由核心組成，核心包含著負負電荷的質子和中性的中子，而圍繞在核心周圍的是帶有負電荷的電子。質子和中子具有正電荷，而電子具有負電荷，因此整個原子是中性的。每種粒子的數量決定了原子的原子序數和種類。</s>'],
 'uris': None,
 'data': None}

In [ ]:
#條件式查詢
collection.query(query_texts=["三原色"],
         n_results=3,
         #where_document={"$contains":"三原色"}
)

{'ids': [['1', '76', '19']],
 'distances': [[0.6995472311973572, 1.1620368957519531, 1.2224584817886353]],
 'metadatas': [[{'type': 'chat'}, {'type': 'chat'}, {'type': 'chat'}]],
 'embeddings': None,
 'documents': [['Human: 三原色是什麼？\n\nAssistant: 三原色是紅色、藍色和黃色。</s>',
   'Human: 將以下句子改寫為第三人稱\n我很焦慮\n\nAssistant: 她很焦慮。</s>',
   'Human: 請提供三個與「道歉」意思相同的動詞。\n\nAssistant: 承認、表達遺憾、彌補。</s>']],
 'uris': None,
 'data': None}

#### 不想使用VectorDB,但需有快速查詢功能
**使用HF Datasets提供的Search index (FAISS)**

參考HF官方文件: https://huggingface.co/docs/datasets/main/en/faiss_es#faiss

**以下安裝FAISS套件，需[重新啟動工作階段]才能載入**

In [37]:
!pip install faiss-cpu #也有GPU版本: faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 26.1 MB/s eta 0:00:00


In [38]:
## 1.Download the DPR model from 🤗 Transformers:

from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch, faiss
torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [ ]:
## 2.Load your dataset and compute the vector representations:
## 使用剛才整理好的yentinglin/TaiwanChat資料集source_ds

ds_with_embeddings = source_ds.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["documents"], return_tensors="pt"))[0][0].numpy()})

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
## 3.Create the index with Dataset.add_faiss_index():

ds_with_embeddings.add_faiss_index(column='embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['documents', 'embeddings'],
    num_rows: 100
})

In [ ]:
## 4.Now you can query your dataset with the embeddings index. Load the DPR Question Encoder, and search for a question with Dataset.get_nearest_examples():

from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

question = "三原色是什麼?"
question_embedding = q_encoder(**q_tokenizer(question, return_tensors="pt"))[0][0].numpy()

scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=10)
retrieved_examples["documents"][0]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'Human: 三原色是什麼？\n\nAssistant: 三原色是紅色、藍色和黃色。</s>'

In [ ]:
## 5.You can access the index with Dataset.get_index() and use it for special operations, e.g. query it using range_search:

faiss_index = ds_with_embeddings.get_index('embeddings').faiss_index
limits, distances, indices = faiss_index.range_search(x=question_embedding.reshape(1, -1), thresh=0.95)

In [ ]:
## 6.When you are done querying, save the index on disk with Dataset.save_faiss_index():

ds_with_embeddings.save_faiss_index('embeddings', 'my_index.faiss')

In [ ]:
## 7.Reload it at a later time with Dataset.load_faiss_index():

ds = load_dataset('yentinglin/TaiwanChat', split='train[:100]')
ds.load_faiss_index('embeddings', 'my_index.faiss')

In [ ]:
#依載入的faiss_index查詢question_embedding最接近的樣本
ds.get_nearest_examples('embeddings', question_embedding, k=3)

NearestExamplesResults(scores=array([ 89.684586, 103.13307 , 103.67672 ], dtype=float32), examples={'conversations': [[{'from': 'human', 'value': '三原色是什麼？'}, {'from': 'gpt', 'value': '三原色是紅色、藍色和黃色。'}], [{'from': 'human', 'value': '將以下句子改寫為第三人稱\n我很焦慮'}, {'from': 'gpt', 'value': '她很焦慮。'}], [{'from': 'human', 'value': '給一個使用以下物體的隱喻的例子\n星星'}, {'from': 'gpt', 'value': '星星在夜空中閃爍如同鑽石。'}]], 'id': ['baize_1', 'baize_76', 'baize_71']})



---



## 參考資料
- Chroma官網 (https://www.trychroma.com/)
- Chroma官方文件 (https://docs.trychroma.com/)
- Chroma Github (https://github.com/chroma-core/chroma)
- HF Datasets提供的Search index (FAISS) (https://huggingface.co/docs/datasets/main/en/faiss_es#faiss)
- Pinecone介紹什麼是向量資料庫 (https://www.pinecone.io/learn/vector-database/)
- Pinecone介紹向量相似度計算 (https://www.pinecone.io/learn/vector-similarity/)
